In [260]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm_notebook as tqdm

In [261]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')
driver = webdriver.Chrome(chrome_options=chrome_options)

In [262]:
movie_name = input("영화 이름을 입력하세요.")

영화 이름을 입력하세요.코코


In [263]:
movie_start_url = "https://movie.naver.com/"
driver.get(movie_start_url)
time.sleep(0.5)
driver.find_element_by_xpath('//*[@id="ipt_tx_srch"]').send_keys(movie_name)
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="jAutoMV"]/ul/li[1]/a').click()
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[5]/a').click()
time.sleep(0.2)

In [264]:
a = driver.find_element_by_xpath('//*[@id="pointAfterListIframe"]')
basic_url = a.get_attribute("src")
start_url = basic_url + '&page=1'

In [265]:
driver.get(start_url)

In [266]:
import math
count = driver.find_element_by_xpath('/html/body/div/div/div[4]/strong/em').text
count = int(count.replace(",", ""))
pages = math.ceil(count/10)
pages

1396

In [267]:
ratings = []
contents =[]
agree = []
disagree = []
date = []

for page in tqdm(range(1, pages)):
    url = basic_url + '&page=' + str(page)
    driver.get(url)
    for i in range(1,11):
        xpath = '/html/body/div/div/div[6]/ul/li[' + str(i) + ']'
        ratings.append(driver.find_element_by_xpath(xpath + '/div[1]/em').text)
        contents.append(driver.find_element_by_xpath(xpath + '/div[2]/p').text)
        agree.append(driver.find_element_by_xpath(xpath + '/div[3]/strong[1]/span').text)
        disagree.append(driver.find_element_by_xpath(xpath + '/div[3]/strong[2]/span').text)
        date.append(driver.find_element_by_xpath(xpath + '/div[2]/dl/dt/em[2]').text)

driver.close()

HBox(children=(IntProgress(value=0, max=1395), HTML(value='')))

In [268]:
df = pd.DataFrame({"ratings" :ratings, "contents" :contents, "agree" :agree, "disagree":disagree, "date":date})
df["movie"] = movie_name
df = df[["movie", "ratings", "contents", "agree", "disagree", "date"]]

In [269]:
df.head(20)

,movie,ratings,contents,agree,disagree,date
0,코코,9,먼저 떠난 우리 아들도 저렇게 멋진데서 잘 지내길..평생 기억할께 꼭 다시 만나자,11012,290,2018.01.12 17:00
1,코코,10,"박평식 : ""따뜻해, 삶을 껴안고 다독이는 죽음"" 박평식 평론가의 평점이 이렇게 공...",3280,192,2018.01.11 09:04
2,코코,10,'신과 함께'와는 격이 다른 저승영화,3131,262,2018.01.11 22:10
3,코코,10,관람객사람이 진정으로 죽는 순간은 사랑하는 사람에게 잊혀졌을때..사랑하는 사람들을 ...,2878,129,2018.01.11 12:14
4,코코,10,내일 죽는다면 오늘 봐야할 영화,2477,186,2018.01.11 09:18
5,코코,10,"사람은 3번 죽는다.숨이 멎는 순간 생물학 적으로 죽고, 장례식에 온 하객들이 떠나...",2011,37,2018.01.12 23:04
6,코코,9,픽사는 우리를 배신하지 않아...,2030,93,2018.01.11 09:09
7,코코,10,마지막에 코코 할머니 손잡고 울면서 기억해줘 노래부를때 정말 심각하게 슬펐네요...ㅠㅠ,1666,37,2018.01.11 16:05
8,코코,10,픽사 역대급 애니다... 평식이도 8점줬다? 긴말 필요없다 이건 영화관 꼭 가서 보...,1718,101,2018.01.11 09:02
9,코코,8,관람객기억해줘 노래 나와서 울고있는데 아들이 옆에서 팝콘 어딨어! 해서 산통깨짐 너...,1528,108,2018.01.12 18:24


In [270]:
df.to_excel(movie_name + ".xlsx")